In [ ]:
import os
import google.generativeai as genai
print(f'You have version {genai.__version__}')

genai.configure(api_key=os.getenv('GEMININ_API_KEY'))

In [21]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from collections.abc import Iterable
from random import randint
from typing import List, TypedDict, Annotated
from pydantic import BaseModel, Field
from pprint import pprint
from IPython.display import Image, display

from langgraph.prebuilt import InjectedState
from langchain_core.messages.tool import ToolMessage
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages.ai import AIMessage
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

In [22]:
# Define the OrderState with more structure
class OrderItem(BaseModel):
    """Represents an item in the customer's order."""
    item: str = Field(description="The name of the item.")
    modifiers: List[str] = Field(default_factory=list, description="A list of modifiers for the item.")
    quantity: int = Field(default=1, description="The quantity of the item.")

class OrderState(TypedDict):
    """State representing the customer's order conversation."""

    # This preserves the conversation history
    messages: Annotated[list, add_messages]

    # The customer's in-progress order.
    order: List[OrderItem]

    # Flag indicating that the order is placed and completed.
    finished: bool


# The system instruction defines how the chatbot is expected to behave and includes rules for when to call different functions, as well as rules for the conversation, such as tone and what is permitted for discussion.
BARISTABOT_SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    "You are a BaristaBot, an interactive cafe ordering system. A human will talk to you about the "
    "available products you have and you will answer any questions about menu items (and only about "
    "menu items - no off-topic discussion, but you can chat about the products and their history). "
    "The customer will place an order for 1 or more items from the menu, which you will structure "
    "and send to the ordering system after confirming the order with the human. "
    "\n\n"
    "Add items to the customer's order with add_to_order, and reset the order with clear_order. "
    "While conversing, you can also inform customers of the total price they have to pay yet"
    "To see the contents of the order so far, call get_order (this is shown to you, not the user) "
    "Always confirm_order with the user (double-check) before calling place_order. Calling confirm_order will "
    "display the order items to the user and returns their response to seeing the list. Their response may contain modifications. "
    "Always verify and respond with drink and modifier names from the MENU before adding them to the order. "
    "If you are unsure a drink or modifier matches those on the MENU, ask a question to clarify or redirect. "
    "You only have the modifiers listed on the menu. "
    "Once the customer has finished ordering items, Call confirm_order to ensure it is correct then make "
    "any necessary updates and then call place_order. Once place_order has returned, thank the user and "
    "say goodbye!",
)

# This is the message with which the system opens the conversation.
WELCOME_MSG = "Welcome to the BaristaBot cafe. Type `q` to quit. How may I serve you today?"

In [23]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

def chatbot(state: OrderState) -> OrderState:
    """The chatbot itself. A simple wrapper around the model's own chat interface."""
    message_history = [BARISTABOT_SYSINT] + state["messages"]
    return {"messages": [llm.invoke(message_history)]}

graph_builder = StateGraph(OrderState)

graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")

chat_graph = graph_builder.compile()

In [ ]:
Image(chat_graph.get_graph().draw_mermaid_png())

In [ ]:
user_msg = "Hello, what can you do?"
state = chat_graph.invoke({"messages": [user_msg]})

# The state object contains lots of information. Uncomment the pprint lines to see it all.
# pprint(state)

# Note that the final state now has 2 messages. Our HumanMessage, and an additional AIMessage.
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")

In [ ]:
user_msg = "Oh great, what kinds of latte can you make?"

state["messages"].append(user_msg)
state = chat_graph.invoke(state)

# pprint(state)
for msg in state["messages"]:
    print(f"{type(msg).__name__}: {msg.content}")

In [ ]:
from langchain_core.messages.ai import AIMessage

def human_node(state: OrderState) -> OrderState:
    """Display the last model message to the user, and receive the user's input."""
    last_msg = state["messages"][-1]
    print("Model:", last_msg.content)

    user_input = input("User: ")

    # If it looks like the user is trying to quit, flag the conversation as over.
    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True

    return state | {"messages": [("user", user_input)]}


def chatbot_with_welcome_msg(state: OrderState) -> OrderState:
    """The chatbot itself. A wrapper around the model's own chat interface."""

    if state["messages"]:
        # If there are messages, continue the conversation with the Gemini model.
        new_output = llm.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        # If there are no messages, start with the welcome message.
        new_output = AIMessage(content=WELCOME_MSG)

    return state | {"messages": [new_output]}


# Start building a new graph.
graph_builder = StateGraph(OrderState)

# Add the chatbot and human nodes to the app graph.
graph_builder.add_node("chatbot", chatbot_with_welcome_msg)
graph_builder.add_node("human", human_node)

# Start with the chatbot again.
graph_builder.add_edge(START, "chatbot")

# The chatbot will always go to the human next.
graph_builder.add_edge("chatbot", "human")

In [ ]:
from typing import Literal

def maybe_exit_human_node(state: OrderState) -> Literal["chatbot", "__end__"]:
    """Route to the chatbot, unless it looks like the user is exiting."""
    if state.get("finished", False):
        return END
    else:
        return "chatbot"


graph_builder.add_conditional_edges("human", maybe_exit_human_node)

chat_with_human_graph = graph_builder.compile()

Image(chat_with_human_graph.get_graph().draw_mermaid_png())

In [ ]:
# Remember that this will loop forever, unless you input `q`, `quit` or one of the other exit terms defined in `human_node`.
state = chat_with_human_graph.invoke({"messages": []})

# Things to try:
 # - Just chat! There's no ordering or menu yet.
#  - 'q' to exit.

# pprint(state)

In [ ]:
from langchain_core.tools import tool

@tool
def get_menu() -> str:
    """Provide the latest up-to-date menu."""
    # Note that this is just hard-coded text, but you could connect this to a live stock
    # database, or you could use Gemini's multi-modal capabilities and take live photos of
    # your cafe's chalk menu or the products on the counter and assmble them into an input.

    return """
    MENU:
    
    Coffee Drinks:
    Espresso - $3.00
    Americano - $3.50
    Cold Brew - $4.00

    Coffee Drinks with Milk:
    Latte - $4.50
    Cappuccino - $4.00
    Cortado - $4.25
    Macchiato - $3.75
    Mocha - $4.75
    Flat White - $4.50

    Tea Drinks:
    English Breakfast Tea - $3.00
    Green Tea - $3.00
    Earl Grey - $3.25

    Tea Drinks with Milk:
    Chai Latte - $4.25
    Matcha Latte - $4.75
    London Fog - $4.00

    Other Drinks:
    Steamer - $3.50
    Hot Chocolate - $4.00

    Modifiers (some may have an extra charge):
    Milk options: Whole, 2%, Oat, Almond, 2% Lactose Free (default: whole)
    Espresso shots: Single, Double, Triple, Quadruple (default: Double)
    Caffeine: Decaf, Regular (default: Regular)
    Hot-Iced: Hot, Iced (default: Hot)
    Sweeteners: Vanilla, Hazelnut, Caramel, Chocolate, Sugar-free Vanilla
    Special requests: Extra hot, One pump, Half caff, Extra foam, etc.

    Soy milk is out of stock today.
    """

In [ ]:
from langgraph.prebuilt import ToolNode

# Define the tools and create a "tools" node.
tools = [get_menu]
tool_node = ToolNode(tools)

llm_with_tools = llm.bind_tools(tools)

def maybe_route_to_tools(state: OrderState) -> Literal["tools", "human"]:
    """Route between human or tool nodes, depending if a tool call is made."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    # Only route based on the last message.
    msg = msgs[-1]

    # When the chatbot returns tool_calls, route to the "tools" node.
    if hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        return "tools"
    else:
        return "human"


def chatbot_with_tools(state: OrderState) -> OrderState:
    """The chatbot with tools. A simple wrapper around the model's own chat interface."""
    defaults = {"order": [], "finished": False}

    if state["messages"]:
        new_output = llm_with_tools.invoke([BARISTABOT_SYSINT] + state["messages"])
    else:
        new_output = AIMessage(content=WELCOME_MSG)

    # Set up some defaults if not already set, then pass through the provided state,
    # overriding only the "messages" field.
    return defaults | state | {"messages": [new_output]}


graph_builder = StateGraph(OrderState)

# Add the nodes, including the new tool_node.
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)

# Chatbot may go to tools, or human.
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
# Human may go back to chatbot, or exit.
graph_builder.add_conditional_edges("human", maybe_exit_human_node)

# Tools always route back to chat afterwards.
graph_builder.add_edge("tools", "chatbot")

graph_builder.add_edge(START, "chatbot")
graph_with_menu = graph_builder.compile()

Image(graph_with_menu.get_graph().draw_mermaid_png())

In [ ]:
# Remember that you have not implemented ordering yet, so this will loop forever,
# unless you input `q`, `quit` or one of the other exit terms defined in the
# `human_node`.
# Uncomment this line to execute the graph:
state = graph_with_menu.invoke({"messages": []})

# Things to try:
# - I'd love an espresso drink, what have you got?
# - What teas do you have?
# - Can you do a long black? (this is on the menu as an "Americano" - see if it can figure it out)
# - 'q' to exit.


# pprint(state)

# Handle Orders
To build up an order during the chat conversation, you will need to update the state to track the order, and provide simple tools that update this state. These need to be explicit as the model should not directly have access to the apps internal state, or it risks being manipulated arbitrarily.

The ordering tools will be added as stubs in a separate node so that you can edit the state directly. Using the @tool annotation is still a handy way to define their schema, so the ordering tools below are implemented as empty Python functions.

In [ ]:
from collections.abc import Iterable
from random import randint
from typing import List

from langgraph.prebuilt import InjectedState
from langchain_core.messages.tool import ToolMessage

# These functions have no body; LangGraph does not allow @tools to update
# the conversation state, so you will implement a separate node to handle
# state updates. Using @tools is still very convenient for defining the tool
# schema, so empty functions have been defined that will be bound to the LLM
# but their implementation is deferred to the order_node.


@tool
def add_to_order(drink: str, modifiers: Iterable[str] = [], quantity: int=  1) -> str:
    """Adds the specified drink to the customer's order, including any modifiers and quantity.

    Returns:
      The updated order in progress.
    """

@tool
def remove_items(item_name: str) -> str:
    """Removes the specified item from the customer's order.

    Returns:
      The updated order in progress.
    """
    
@tool
def confirm_order() -> str:
    """Asks the customer if the order is correct.

    Returns:
      The user's free-text response.
    """
    
@tool
def get_order() -> str:
    """Returns the users order so far. One item per line."""

@tool
def clear_order():
    """Removes all items from the user's order."""

@tool
def place_order() -> int:
    """Sends the order to the barista for fulfillment.

    Returns:
      The estimated number of minutes until the order is ready.
    """

# Function to add an item to the order
def add_to_order(state: OrderState, item: str, modifiers: List[str] = None, quantity: int = 1) -> OrderState:
    modifiers = modifiers or []
    state["order"].append({"item": item, "modifiers": modifiers, "quantity": quantity})
    return state

# Function to remove an item from the order
def remove_items(state: OrderState, item_name: str) -> OrderState:
    state["order"] = [item for item in state["order"] if item["item"] != item_name]
    return state

# Function to clear the order
def clear_order(state: OrderState) -> OrderState:
    state["order"] = []
    return state

# Function to get the current order summary
def get_order_summary(state: OrderState) -> str:
    if not state["order"]:
        return "(no order)"
    return "\n".join(
        f'{item["quantity"]}x {item["item"]} ({", ".join(item["modifiers"]) if item["modifiers"] else "no modifiers"})'
        for item in state["order"]
    )
    
def order_node(state: OrderState) -> OrderState:
    """The ordering node. This is where the order state is manipulated."""
    tool_msg = state.get("messages", [])[-1]
    order = state.get("order", [])
    outbound_msgs = []
    order_placed = False

    for tool_call in tool_msg.tool_calls:

        if tool_call["name"] == "add_to_order":

           order.append({
               "item": tool_call["args"]["drink"],
                "modifiers": tool_call["args"].get("modifiers", []),
                "quantity": tool_call["args"].get("quantity", 1),
           })
           response = get_order_summary(state)
           
        elif tool_call["name"] == "remove_items":
            item_name = tool_call["args"]["drink"]
            state["order"] = [item for item in state["order"] if item["item"].lower() != item_name.lower()]
            response = f'Removed {item_name} from the order.\n' + get_order_summary(state)

        elif tool_call["name"] == "confirm_order":
            # We could entrust the LLM to do order confirmation, but it is a good practice to
            # show the user the exact data that comprises their order so that what they confirm
            # precisely matches the order that goes to the kitchen - avoiding hallucination
            # or reality skew.

            # In a real scenario, this is where you would connect your POS screen to show the
            # order to the user.
            
            response = get_order_summary(state)

        elif tool_call["name"] == "get_order":

            response = get_order_summary(state)

        elif tool_call["name"] == "clear_order":
            state = clear_order(state)
            response = "Order cleared."
            
        elif tool_call["name"] == "place_order":
            order_text = get_order_summary(state)
            print("Sending order to kitchen!")
            print(order_text)
            order_placed = True
            response = f"Your order has been placed! Estimated wait time: {randint(1, 5)} minutes."
        else:
            raise NotImplementedError(f'Unknown tool call: {tool_call["name"]}')

        # Record the tool results as tool messages.
        outbound_msgs.append(
            ToolMessage(
                content=response,
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outbound_msgs, "order": order, "finished": order_placed}


def maybe_route_to_tools(state: OrderState) -> str:
    """Route between chat and tool nodes if a tool call is made."""
    if not (msgs := state.get("messages", [])):
        raise ValueError(f"No messages found when parsing state: {state}")

    msg = msgs[-1]

    if state.get("finished", False):
        # When an order is placed, exit the app. The system instruction indicates
        # that the chatbot should say thanks and goodbye at this point, so we can exit
        # cleanly.
        return END

    elif hasattr(msg, "tool_calls") and len(msg.tool_calls) > 0:
        # Route to `tools` node for any automated tool calls first.
        if any(
            tool["name"] in tool_node.tools_by_name.keys() for tool in msg.tool_calls
        ):
            return "tools"
        else:
            return "ordering"

    else:
        return "human"

In [ ]:
# Auto-tools will be invoked automatically by the ToolNode
auto_tools = [get_menu]
tool_node = ToolNode(auto_tools)

# Order-tools will be handled by the order node.
order_tools = [add_to_order, confirm_order, get_order, clear_order, place_order]

# The LLM needs to know about all of the tools, so specify everything here.
llm_with_tools = llm.bind_tools(auto_tools + order_tools)

graph_builder = StateGraph(OrderState)

# Nodes
graph_builder.add_node("chatbot", chatbot_with_tools)
graph_builder.add_node("human", human_node)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("ordering", order_node)

# Chatbot -> {ordering, tools, human, END}
graph_builder.add_conditional_edges("chatbot", maybe_route_to_tools)
# Human -> {chatbot, END}
graph_builder.add_conditional_edges("human", maybe_exit_human_node)

# Tools (both kinds) always route back to chat afterwards.
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("ordering", "chatbot")

graph_builder.add_edge(START, "chatbot")
graph_with_order_tools = graph_builder.compile()

Image(graph_with_order_tools.get_graph().draw_mermaid_png())

In [ ]:
# The default recursion limit for traversing nodes is 25 - setting it higher
# means you can try a more complex order with multiple steps and round-trips.
config = {"recursion_limit": 100}

# Uncomment this line to execute the graph:
state = graph_with_order_tools.invoke({"messages": []}, config)

# Things to try:
# - Order a drink!
# - Make a change to your order.
# - "Which teas are from England?"
# - Note that the graph should naturally exit after placing an order.

# pprint(state)

In [ ]:
# Uncomment this once you have run the graph from the previous cell.
pprint(state["order"])